In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# API endpoint
url = "https://api.data.gov.sg/v1/environment/rainfall"

time = [(2022,9,30),(2022,10,31),(2022,11,30),(2022,12,31),(2023,1,31),(2023,2,28)]
#time = [(2022,9,1)]
for t in time:
# Start and end dates
    start_date = datetime(t[0], t[1], 1)
    end_date = datetime(t[0], t[1], t[2], 23)  # Include all hours on the last day

    # Initialize current date
    current_date = start_date

    # Storage for results
    s108_readings = []

    # Loop through every hour in September 2022
    while current_date <= end_date:
        # Format the date_time as ISO 8601
        date_time = current_date.strftime("%Y-%m-%dT%H:00:00")
        params = {"date_time": date_time}
        
        # Request data from the API
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            # Extract readings
            items = data.get("items", [])
            for item in items:
                readings = item.get("readings", [])
                for reading in readings:
                    # Check for S108
                    if reading["station_id"] == "S108":
                        # Save the timestamp and wind speed value
                        s108_readings.append({
                            "timestamp": pd.to_datetime(item["timestamp"]),
                            "value": reading["value"]
                        })
        else:
            print(f"Error fetching data for {date_time}: {response.status_code}")
        
        # Increment by 1 hour
        current_date += timedelta(hours=1)

    # Convert the results into a pandas DataFrame
    df = pd.DataFrame(s108_readings)

    # Convert timestamp to pandas datetime for easier handling
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Display the first few rows of the DataFrame
    print(df.head())

    # Save to CSV for future use
    df.to_csv(f"Rainfall_Raw_{t[0]}_{t[1]}.csv", index=False)
    df.set_index('timestamp', inplace=True)
    new_df=df.resample('6H').mean()
    new_df.index = pd.to_datetime(new_df.index)
    new_df.to_csv(f"Rainfall_6H_{t[0]}_{t[1]}.csv", index=False)



KeyboardInterrupt: 